In [47]:
import os
import re

import pandas as pd
import anndata
import scanpy as sc
import mygene

import mgitools.os_helpers as os_helpers

In [2]:
clinical = pd.read_csv('../data/cptac/PDC_clinical_manifest_10252021_110609.csv')
clinical

,Case ID,Cases Submitter ID,Genomic and Imaging Data Resource,Ethnicity,Gender,Race,Morphology,Primary Diagnosis,Site of Resection or Biopsy,Tissue or Organ of Origin,...,New Event Anatomic Site,New Event Type,Overall Survival,Perineural Invasion Present,Prior Treatment,Progression Free Survival,Progression Free Survival Event,Residual Disease,Vascular Invasion Present,Year of Diagnosis
0,0d000f9c-f76f-42a0-b88b-9a9054919adb,C3N-02589,GDC: https://portal.gdc.cancer.gov/cases/12126...,Not Reported,Male,Not Reported,8500/3,"Infiltrating duct carcinoma, NOS",Head of pancreas,Head of pancreas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R0,NaN,2016.0
1,588c0d27-fc63-4f2b-904a-09cd2507fc00,C3L-04473,GDC: https://portal.gdc.cancer.gov/cases/dbc7d...,Not Reported,Female,Not Reported,8500/3,"Infiltrating duct carcinoma, NOS","Pancreas, NOS","Pancreas, NOS",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,d21fa2fa-9a2d-4baa-b7d8-601b906a16c7,C3N-03069,GDC: https://portal.gdc.cancer.gov/cases/20843...,Not Reported,Female,Not Reported,8500/3,"Infiltrating duct carcinoma, NOS",Body of pancreas,Body of pancreas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R0,NaN,2016.0
3,9e815dee-cfa4-46a0-a284-741ca0547c92,C3L-03639,GDC: https://portal.gdc.cancer.gov/cases/0b2f0...,Not Reported,Male,Not Reported,8500/3,"Infiltrating duct carcinoma, NOS",Head of pancreas,Head of pancreas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7e9a404c-2b80-4ea6-aa7c-7ebc02493cdc,C3N-01714,GDC: https://portal.gdc.cancer.gov/cases/fa3bd...,Not Reported,Male,Not Reported,8500/3,"Infiltrating duct carcinoma, NOS",Head of pancreas,Head of pancreas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R0,NaN,2017.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,ca63469f-6f24-47be-8333-ebd44c3f831d,C3N-00513,GDC: https://portal.gdc.cancer.gov/cases/acb1f...,Not Reported,Male,Not Reported,8500/3,"Infiltrating duct carcinoma, NOS",Head of pancreas,Head of pancreas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R0,NaN,2017.0
140,c531b8d3-736b-4f87-b4f5-0234f5475cf5,C3N-03839,GDC: https://portal.gdc.cancer.gov/cases/2525b...,Not Reported,Female,Not Reported,8500/3,"Infiltrating duct carcinoma, NOS",Head of pancreas,Head of pancreas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RX,NaN,2017.0
141,dc5c98c3-f062-4a93-917f-1854af37c3f4,C3N-00511,GDC: https://portal.gdc.cancer.gov/cases/2d562...,Not Reported,Female,Not Reported,8500/3,"Infiltrating duct carcinoma, NOS",Head of pancreas,Head of pancreas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RX,NaN,2016.0
142,d9465f75-1a21-4c4f-8b6f-14e0600dffda,C3N-03840,GDC: https://portal.gdc.cancer.gov/cases/5ee53...,Not Reported,Female,Not Reported,8500/3,"Infiltrating duct carcinoma, NOS",Body of pancreas,Body of pancreas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
fps = sorted(os_helpers.listfiles('../data/cptac/ALL_RNA-Seq_Expr_WashU_readcount_fpkm_individual/'))
fps[:2], len(fps)

(['../data/cptac/ALL_RNA-Seq_Expr_WashU_readcount_fpkm_individual/01BR001.RNA-Seq.T.hg38.tsv.gz',
  '../data/cptac/ALL_RNA-Seq_Expr_WashU_readcount_fpkm_individual/01BR008.RNA-Seq.T.hg38.tsv.gz'],
 1471)

In [9]:
pool = set(clinical['Cases Submitter ID'])
fps = [fp for fp in fps
       if fp.split('/')[-1].split('.')[0] in pool
       if '.T' in fp]
len(fps)

140

In [11]:
df = pd.read_csv(fps[0], sep='\t')
df

,gene_id,read_count,fpkm,fpkm_uq
0,ENSG00000223972.5,8,0.128497,2.540469e+03
1,ENSG00000227232.5,63,1.299535,2.569263e+04
2,ENSG00000278267.1,4,1.639283,3.240966e+04
3,ENSG00000243485.3,0,0.000000,0.000000e+00
4,ENSG00000274890.1,0,0.000000,0.000000e+00
...,...,...,...,...
60478,ENSG00000198695.2,145,7.696825,1.521711e+05
60479,ENSG00000210194.1,6,2.423288,4.790993e+04
60480,ENSG00000198727.2,19836,484.475000,9.578375e+06
60481,ENSG00000210195.2,70,29.556773,5.843560e+05


In [20]:
filtered = pd.DataFrame.from_dict({'file_path': fps})
filtered.index = [x.split('/')[-1].split('.')[0] for x in fps]
filtered

,file_path
C3L-00017,../data/cptac/ALL_RNA-Seq_Expr_WashU_readcount...
C3L-00102,../data/cptac/ALL_RNA-Seq_Expr_WashU_readcount...
C3L-00189,../data/cptac/ALL_RNA-Seq_Expr_WashU_readcount...
C3L-00277,../data/cptac/ALL_RNA-Seq_Expr_WashU_readcount...
C3L-00401,../data/cptac/ALL_RNA-Seq_Expr_WashU_readcount...
...,...
C3N-04119,../data/cptac/ALL_RNA-Seq_Expr_WashU_readcount...
C3N-04126,../data/cptac/ALL_RNA-Seq_Expr_WashU_readcount...
C3N-04282,../data/cptac/ALL_RNA-Seq_Expr_WashU_readcount...
C3N-04283,../data/cptac/ALL_RNA-Seq_Expr_WashU_readcount...


In [21]:
import numpy as np
gene_order = df['gene_id'].to_list()
def get_expression_df(df, gene_order):
    expression_df = None
    for i, (case_id, fp) in enumerate(zip(filtered.index, filtered['file_path'])):
        if i % 10 == 0: print(i)

        temp = pd.read_csv(fp, sep='\t')
        temp = temp.set_index('gene_id')
        temp = temp[['read_count']]
        temp.columns = [case_id]
        temp = temp.loc[gene_order]

        if expression_df is None:
            expression_df = temp.copy()
        else:
            expression_df = pd.concat((expression_df, temp), axis=1)
            
    return expression_df

def convert_to_hugo(expression_df):
    expression_df.index = [x.split('.')[0] for x in expression_df.index]
    expression_df.index.name = 'gene_id'
    expression_df = expression_df.groupby('gene_id').mean()
    
    mg = mygene.MyGeneInfo()
    genes = list(expression_df.index)
    
    results = mg.querymany(genes, scopes='ensemblgene', fields='symbol', species='human')
    gene_id_to_symbol = {d['query']: d['symbol'] for d in results if 'symbol' in d}
    
    mask = np.asarray([True if gene_id in gene_id_to_symbol else False for gene_id in expression_df.index])
    expression_df = expression_df[mask]
    expression_df['gene_symbol'] = [gene_id_to_symbol[gene_id] for gene_id in expression_df.index]
    
    expression_df = expression_df.set_index('gene_symbol')
    expression_df.index.name = 'gene'
    expression_df = expression_df.groupby('gene').mean()
    
    return expression_df

In [22]:
expression_df = get_expression_df(filtered, gene_order)
expression_df = convert_to_hugo(expression_df)
expression_df

0
10
20
30
40
50
60
70
80
90
100
110
120
130
querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-20000...done.
querying 20001-21000...done.
querying 21001-22000...done.
querying 22001-23000...done.
querying 23001-24000...done.
querying 24001-25000...done.
querying 25001-26000...done.
querying 26001-27000...done.
querying 27001-28000...done.
querying 28001-29000...done.
querying 29001-30000...done.
querying 30001-31000...done.
querying 31001-32000...done.
querying 32001-33000...done.
querying 33001-34000

,C3L-00017,C3L-00102,C3L-00189,C3L-00277,C3L-00401,C3L-00589,C3L-00598,C3L-00599,C3L-00622,C3L-00625,...,C3N-03780,C3N-03839,C3N-03840,C3N-03853,C3N-03884,C3N-04119,C3N-04126,C3N-04282,C3N-04283,C3N-04284
gene,,,,,,,,,,,,,,,,,,,,,
5S_rRNA,0.333333,0.000000,0.666667,0.000000,0.000000,1.000000,1.333333,0.000000,1.666667,1.333333,...,0.000000,0.000000,0.333333,0.333333,0.000000,0.0,0.666667,0.000000,0.000000,0.000000
5_8S_rRNA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,39.666667,0.000000,0.000000,...,0.333333,0.000000,0.000000,0.000000,0.666667,0.0,0.000000,0.000000,0.000000,0.000000
7SK,218590.714286,3137.285714,171174.714286,11427.714286,201480.428571,244936.285714,11765.857143,241532.285714,195775.285714,196375.571429,...,55480.000000,220094.571429,147672.285714,9756.571429,57249.285714,167732.0,171055.428571,204093.571429,4873.857143,23586.428571
A1BG,5.000000,6.000000,15.000000,20.000000,4.000000,11.000000,16.000000,15.000000,18.000000,15.000000,...,16.000000,6.000000,8.000000,14.000000,7.000000,9.0,15.000000,8.000000,12.000000,10.000000
A1BG-AS1,96.000000,87.000000,216.000000,101.000000,110.000000,101.000000,56.000000,135.000000,193.000000,158.000000,...,88.000000,95.000000,116.000000,43.000000,56.000000,179.0,171.000000,128.000000,105.000000,71.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZEF1,3920.000000,3634.000000,3834.000000,5639.000000,4243.000000,4098.000000,3732.000000,4071.000000,5003.000000,3637.000000,...,2342.000000,2890.000000,4482.000000,3094.000000,1601.000000,3577.0,4100.000000,3750.000000,2693.000000,5553.000000
ZZZ3,3530.000000,3375.000000,2824.000000,6274.000000,3455.000000,3838.000000,2760.000000,3307.000000,3411.000000,2942.000000,...,2211.000000,4217.000000,3738.000000,2952.000000,1652.000000,3801.0,4865.000000,5332.000000,5136.000000,3367.000000
hsa-mir-1253,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000


In [23]:
expression_df.to_csv('../data/cptac/read_counts.txt', sep='\t', header=True, index=True)

In [42]:
# cpm
cpm = pd.DataFrame(data=(expression_df.values / np.sum(expression_df.values, axis=0)) * 1000000,
                   columns=expression_df.columns, index=expression_df.index)
cpm

,C3L-00017,C3L-00102,C3L-00189,C3L-00277,C3L-00401,C3L-00589,C3L-00598,C3L-00599,C3L-00622,C3L-00625,...,C3N-03780,C3N-03839,C3N-03840,C3N-03853,C3N-03884,C3N-04119,C3N-04126,C3N-04282,C3N-04283,C3N-04284
gene,,,,,,,,,,,,,,,,,,,,,
5S_rRNA,0.008628,0.000000,0.018643,0.000000,0.000000,0.024259,0.036450,0.000000,0.039521,0.040922,...,0.000000,0.000000,0.009492,0.012703,0.000000,0.000000,0.016583,0.000000,0.000000,0.000000
5_8S_rRNA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.146391,0.000000,0.000000,...,0.017542,0.000000,0.000000,0.000000,0.037177,0.000000,0.000000,0.000000,0.000000,0.000000
7SK,5658.082013,84.475985,4786.914823,276.880698,6134.984783,5941.980369,321.648677,6980.432115,4642.363607,6027.032848,...,2919.752448,5537.599589,4204.958442,371.824384,3192.562369,4564.491107,4254.934769,4823.867529,123.608348,705.332957
A1BG,0.129422,0.161559,0.419476,0.484578,0.121798,0.266852,0.437399,0.433509,0.426829,0.460370,...,0.842034,0.150961,0.227799,0.533542,0.390362,0.244917,0.373119,0.189085,0.304338,0.299042
A1BG-AS1,2.484899,2.342602,6.040458,2.447117,3.349449,2.450188,1.530898,3.901583,4.576554,4.849234,...,4.631186,2.390209,3.303092,1.638736,3.122895,4.871127,4.253556,3.025353,2.662958,2.123197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZEF1,101.466714,97.850741,107.218122,136.626644,129.197365,99.414570,102.023409,117.654412,118.634715,111.624467,...,123.252708,72.712665,127.624650,117.912799,89.281330,97.340905,101.985846,88.633381,68.298531,166.057947
ZZZ3,91.371811,90.876789,78.973390,152.011982,105.203134,93.107155,75.451396,95.574341,80.884072,90.293973,...,116.358556,106.100106,106.439300,112.501158,92.125395,103.436617,121.014912,126.024850,130.256685,100.687396
hsa-mir-1253,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [48]:
cpm.columns = [x.replace('-', '_') for x in cpm.columns]

In [50]:
cpm

,C3L_00017,C3L_00102,C3L_00189,C3L_00277,C3L_00401,C3L_00589,C3L_00598,C3L_00599,C3L_00622,C3L_00625,...,C3N_03780,C3N_03839,C3N_03840,C3N_03853,C3N_03884,C3N_04119,C3N_04126,C3N_04282,C3N_04283,C3N_04284
gene,,,,,,,,,,,,,,,,,,,,,
5S_rRNA,0.008628,0.000000,0.018643,0.000000,0.000000,0.024259,0.036450,0.000000,0.039521,0.040922,...,0.000000,0.000000,0.009492,0.012703,0.000000,0.000000,0.016583,0.000000,0.000000,0.000000
5_8S_rRNA,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.146391,0.000000,0.000000,...,0.017542,0.000000,0.000000,0.000000,0.037177,0.000000,0.000000,0.000000,0.000000,0.000000
7SK,5658.082013,84.475985,4786.914823,276.880698,6134.984783,5941.980369,321.648677,6980.432115,4642.363607,6027.032848,...,2919.752448,5537.599589,4204.958442,371.824384,3192.562369,4564.491107,4254.934769,4823.867529,123.608348,705.332957
A1BG,0.129422,0.161559,0.419476,0.484578,0.121798,0.266852,0.437399,0.433509,0.426829,0.460370,...,0.842034,0.150961,0.227799,0.533542,0.390362,0.244917,0.373119,0.189085,0.304338,0.299042
A1BG-AS1,2.484899,2.342602,6.040458,2.447117,3.349449,2.450188,1.530898,3.901583,4.576554,4.849234,...,4.631186,2.390209,3.303092,1.638736,3.122895,4.871127,4.253556,3.025353,2.662958,2.123197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZZEF1,101.466714,97.850741,107.218122,136.626644,129.197365,99.414570,102.023409,117.654412,118.634715,111.624467,...,123.252708,72.712665,127.624650,117.912799,89.281330,97.340905,101.985846,88.633381,68.298531,166.057947
ZZZ3,91.371811,90.876789,78.973390,152.011982,105.203134,93.107155,75.451396,95.574341,80.884072,90.293973,...,116.358556,106.100106,106.439300,112.501158,92.125395,103.436617,121.014912,126.024850,130.256685,100.687396
hsa-mir-1253,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [49]:
cpm.to_csv('../data/cptac/cpm.txt', sep='\t', header=True, index=True, float_format='%.2f')

In [38]:
c = pd.read_csv('../data/bulk_rna_seq/bailey/counts.txt', sep='\t', index_col=0)
c

,SA518851,SA528771,SA528766,SA528763,SA518817,SA518765,SA518854,SA518806,SA528767,SA518750,...,SA412299,SA411841,SA411797,SA411769,SA411833,SA411430,SA411406,SA411923,SA411305,SA411682
Gene,,,,,,,,,,,,,,,,,,,,,
TSPAN6,951,71,26,191,845,885,785,896,225,364,...,3130,1009,790,701,614,490,935,1040,1107,1771
TNMD,0,0,0,0,0,23,0,0,0,4,...,1,0,3,0,1,0,0,0,1,0
DPM1,1314,107,16,126,687,1581,765,471,116,450,...,984,1293,1706,496,997,1268,631,770,1594,1008
SCYL3,265,237,43,192,643,904,651,565,383,582,...,1373,235,886,470,1456,381,766,278,646,244
C1orf112,443,113,22,102,255,450,358,193,243,234,...,383,525,416,184,452,205,223,258,611,820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC008735.3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
AC245884.10,1,4,0,4,6,10,2,8,9,36,...,0,0,2,0,4,2,3,4,0,1
AC008753.3,0,0,0,0,0,1,0,2,0,0,...,0,0,1,0,0,0,0,0,0,0


In [39]:
pd.DataFrame(data=(c.values / np.sum(c.values, axis=0)) * 1000000,
                   columns=c.columns, index=c.index).sort_values('Gene')

,SA518851,SA528771,SA528766,SA528763,SA518817,SA518765,SA518854,SA518806,SA528767,SA518750,...,SA412299,SA411841,SA411797,SA411769,SA411833,SA411430,SA411406,SA411923,SA411305,SA411682
Gene,,,,,,,,,,,,,,,,,,,,,
A1BG,1.873957,1.781443,1.851646,1.125449,1.267181,0.842934,0.963219,0.967079,2.390215,1.163613,...,3.788088,0.131573,1.538798,1.456038,0.883073,1.269928,1.874871,2.855518,1.305073,0.216444
A1BG-AS1,1.117167,0.950103,0.925823,2.077753,1.267181,0.991687,0.963219,0.511983,2.390215,0.775742,...,1.638818,0.000000,1.471894,2.053387,1.527478,0.893653,2.879267,3.824354,1.305073,0.270556
A1CF,0.000000,3.206598,3.240381,6.406404,3.225552,1.661076,1.083622,0.113774,8.795990,6.119745,...,2.471660,0.000000,7.292564,29.643447,2.577619,6.584809,2.644908,2.498578,1.334733,0.108222
A2M,246.677592,140.021441,97.674332,318.155870,461.791496,763.252002,921.399501,228.799406,427.083540,887.750865,...,236.043574,0.000000,245.070277,613.029475,689.298322,647.710067,960.637274,647.641643,359.784760,2.326777
A2M-AS1,0.468489,1.662680,0.925823,1.991180,0.767989,2.181712,1.444829,2.673688,2.198997,4.223486,...,0.134329,0.131573,1.471894,1.082695,0.883073,0.235172,2.109230,0.866854,0.593215,0.216444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXDC,27.136337,50.236700,54.160649,31.339435,28.031581,32.899221,41.619099,55.407912,40.538039,41.631503,...,39.949556,11.622255,40.142552,34.198234,29.356218,33.770665,38.100067,43.903587,37.906424,14.988775
ZYG11A,0.396414,4.275464,4.166204,2.337472,0.844787,0.619804,1.163890,1.080852,6.692601,1.077420,...,0.268659,0.043858,0.802851,0.709352,0.668272,0.282206,0.401758,1.376768,0.474572,4.815888
ZYG11B,76.435818,48.336494,51.846091,67.267241,75.685268,74.004652,77.338481,73.497969,64.248968,121.791541,...,53.543688,41.664688,56.132668,64.849708,78.307115,54.795021,59.594129,84.084802,119.147187,69.586877
